In [ ]:
inshp  = '/mnt/d/a3land2023/rule3_4/sample.shp'

dist = 500

angle_min = 0
angle_max = 45

angles_int = 45

outshp = '/mnt/d/a3land2023/rule3_4/cheese_buffer4.shp'
outbf = '/mnt/d/a3land2023/rule3_4/regularbuffer.shp'
outcheese = '/mnt/d/a3land2023/rule3_4/extracheese.shp'

In [ ]:
import geopandas as gp
import numpy as np
from glass.gp.prox.bfing.obj import geodf_buffer
from glass.rd.shp import shp_to_obj
from glass.prop.prj import get_epsg
from glass.gobj import create_polygon
from glass.pd.dagg import col_listwlist_to_row
from glass.pd import merge_df
from glass.wt.shp import df_to_shp
from glass.it.pd import df_to_geodf

In [ ]:
epsg = get_epsg(inshp)

pdf = shp_to_obj(inshp)

In [ ]:
# Get Regular Buffer
rbf = geodf_buffer(pdf, dist)

In [ ]:
rbf['ofeatid'] = rbf.index

In [ ]:
# Get Cheese Buffer
def run_cheese_buffer(r):
    multipoly = []
    
    minangle = 0
    maxangle = angles_int
    
    while maxangle <= 360:
        coords = [(r.x, r.y)]
        for i in range(minangle, maxangle + 1):
            y = r.y + ((r.dist + dist) * np.cos(np.radians(i)))
            x = r.x + ((r.dist + dist) * np.sin(np.radians(i)))
    
            coords.append((x, y))
        
        coords.append((r.x, r.y))   
    
        poly = create_polygon(coords, api='shapely')
        
        multipoly.append([r.featid, poly, f"{str(minangle)} - {str(maxangle)}"])
        
        minangle += angles_int
        maxangle += angles_int
    
    r["geoms"] = multipoly
    
    return r

In [ ]:
odf = pdf.copy()

s = pdf.geometry

t = gp.GeoSeries(gp.points_from_xy(s.envelope.bounds.maxx, s.envelope.bounds.maxy))
pdf["dist"] = s.envelope.centroid.distance(t)

pdf['x'] = s.envelope.centroid.x
pdf['y'] = s.envelope.centroid.y

pdf["featid"] = pdf.index

pdf = pdf.apply(lambda x: run_cheese_buffer(x), axis=1)

pdf.drop(["x", "y", "geometry"], axis=1, inplace=True)

In [ ]:
pdf = col_listwlist_to_row(
    pdf, "geoms", ["ofid", "geom", "direction"],
    geomcol="geom", epsg=epsg
)

In [ ]:
df_to_shp(pdf, outcheese)
df_to_shp(rbf, outbf)

In [ ]:
dfs = []
for i, row in odf.iterrows():
    # Get slices
    slices = pdf[pdf.ofid == i]
    
    # Get regular buffer
    regbf = rbf[rbf.ofeatid == i]
    regbf = regbf.to_dict(orient="records")
    
    bfgeom = regbf[0]["geometry"]
    
    # Get Intersection
    slices["geometry"] = slices.geom.intersection(bfgeom)
    
    dfs.append(slices)

In [ ]:
fdf = merge_df(dfs)
fdf = df_to_geodf(fdf, 'geometry', epsg)

In [ ]:
fdf

In [ ]:
fdf.drop('geom', axis=1, inplace=True)
df_to_shp(fdf, outshp)